In [9]:
import numpy as np
import pandas as pd
import time
import multiprocessing
from multiprocessing import Process, Pipe
from datetime import datetime

In [4]:
data = pd.read_csv('20K_Datapoints.csv')
data

,X,Y
0,0.387717,10.746281
1,4.377074,227.190272
2,0.436813,22.670695
3,3.278660,177.120957
4,5.734753,373.677893
...,...,...
19995,8.228528,744.944460
19996,3.036068,137.230401
19997,2.874888,112.618173
19998,3.993018,236.503574


In [5]:
X = data.iloc[:, 0].to_numpy()
Y = data.iloc[:, 1].to_numpy()

In [13]:
def gradD(connection):
    a=0
    b=0
    c=0
    
    L=0.0001
    epochs=10000
    X=connection.recv()
    Y=connection.recv()
    n=float(len(X))
    
    for i in range(epochs): 
        Y_pred = a*X*X + b*X + c 
        D_a = (-2/n) * sum(X*X * (Y - Y_pred))  
        D_b = (-2/n) * sum(X * (Y - Y_pred))  
        D_c = (-2/n) * sum(Y - Y_pred)  
        a = a - L * D_a
        b = b - L * D_b 
        c = c - L * D_c
    
    connection.send(a)
    connection.send(b)
    connection.send(c)
    connection.close()

In [14]:
if __name__ == '__main__':
    startTime = time.time()
    x1 = data.iloc[0:6666,0]
    x2 = data.iloc[6666:13332,0]
    x3 = data.iloc[13332:19998,0]
    y1 = data.iloc[0:6666,1]
    y2 = data.iloc[6666:13332,1]
    y3 = data.iloc[13332:19998,1]
    
    #start = datetime.now()
    
    parent,child = Pipe()

    process1 = Process(target=gradD, args=(child,))
    process2 = Process(target=gradD, args=(child,))
    process3 = Process(target=gradD, args=(child,))
   
    process1.start()
    parent.send(x1)
    parent.send(y1)
    a1 = parent.recv()
    b1 = parent.recv()
    c1 = parent.recv()
    
    process2.start()
    parent.send(x2)
    parent.send(y2)
    a2 = parent.recv()
    b2 = parent.recv()
    c2 = parent.recv()
    
    process3.start()
    parent.send(x3)
    parent.send(y3)
    a3 = parent.recv()
    b3 = parent.recv()
    c3 = parent.recv()
    
    a = (a1+a2+a3)/3
    b = (b1+b2+b3)/3
    c = (c1+c2+c3)/3
    
    process1.join()
    process2.join()
    process3.join()
    
    stop = datetime.now()
    
    print("A: ", a, "B: ", b, "C: ", c)
    print('That took {} seconds'.format(time.time() - startTime))

A:  9.50251761386896 B:  11.858581743984653 C:  10.756355103429252
That took 93.82001614570618 seconds
